<a href="https://colab.research.google.com/github/niranjan-sa/DataScience/blob/master/Copy_of_Innoplexus_LSTM_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.listdir('drive/')
train = pd.read_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/train_F3WbcTw.csv')
test = pd.read_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/test_tOlRoBf.csv')

In [4]:
train.head(3)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


In [0]:
data = train.append(test, ignore_index=True, sort=False)

In [7]:
data.head(2)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2.0
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2.0


In [8]:
data.tail(2)

,unique_hash,text,drug,sentiment
8201,9936efcb83eded79fe9454df188edd7b96c6109e,It sounds like you're doing very well FG! I wo...,stelara,NaN
8202,05402df12d6769d7f38ab40e0b81464e65e1df0a,"Hi @rambles too! It’s such a hard decision, is...",cladribine,NaN


In [0]:
data = data[['text', 'sentiment']]

In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 0].size)
print(data[ data['sentiment'] == 1].size)
print(data[ data['sentiment'] == 2].size)

1234
1674
7650


In [0]:
max_fatures = 500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [12]:
embed_dim = 128//4
lstm_out = 196//4

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['categorical_accuracy'])
print(model.summary())

W0727 06:53:41.706969 140719304587136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 06:53:41.728445 140719304587136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 06:53:41.731806 140719304587136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 06:53:41.756074 140719304587136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 06:53:41.768598 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18891, 32)         16000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 18891, 32)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 49)                16072     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 150       
Total params: 32,222
Trainable params: 32,222
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
train = X[:5279,:]
test = X[5279:, :]

In [0]:
ytrain = data['sentiment'].iloc[:5279]

In [15]:
Y = pd.get_dummies(ytrain).values
X_train, X_test, Y_train, Y_test = train_test_split(train,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3695, 18891) (3695, 3)
(1584, 18891) (1584, 3)


In [16]:
import gc
del train, X
gc.collect()

24

In [17]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 1)

W0727 06:53:42.861093 140719304587136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
3695/3695 [==============================] - 4544s 1s/step - loss: 0.8313 - categorical_accuracy: 0.7196
Epoch 2/2
3695/3695 [==============================] - 4596s 1s/step - loss: 0.7680 - categorical_accuracy: 0.7261


In [0]:
del X_train, Y_train

In [19]:
import gc
gc.collect()

10

In [20]:
model.history

In [36]:
pred = model.predict(X_test)
labels = pred.argmax(axis=1)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print (accuracy_score(Y_test.argmax(axis=1), labels))
print (confusion_matrix(Y_test.argmax(axis=1), labels))
pd.Series(labels).value_counts()

0.7209595959595959
[[   0    0  173]
 [   0    0  269]
 [   0    0 1142]]


2    1584
dtype: int64

In [35]:
test_op =  model.predict(X_test, verbose=1)

1584/1584 [==============================] - 711s 449ms/step
